## 14.Transformer（2017年）

**学习目标**

1. 熟悉Transformer的编码器-解码器架构

2. 能用代码实现Transformer编码器

3. 能用代码实现Transformer解码器

4. 能用代码实现Transformer编码器-解码器

****

Transformer 是一种深度学习模型，由 Vaswani 等人在 2017 年的论文《Attention Is All You Need》中首次提出。它在自然语言处理（NLP）领域取得了革命性的进展。以下是 Transformer 模型的几个关键特点：

自注意力机制（Self-Attention）：Transformer 摒弃了传统的循环神经网络（RNN）结构，使用自注意力机制来捕捉序列内不同位置之间的依赖关系。这种机制允许模型同时处理序列中的所有元素，从而提高了计算效率。

并行处理能力：由于自注意力机制的特性，Transformer 可以并行处理序列中的所有元素，这与传统的序列模型（如 LSTM 或 GRU）相比，大大提高了训练速度。

Transformer的编码器-解码器架构

Transformer 模型通常由编码器（Encoder）和解码器（Decoder）组成。编码器将输入序列转换为连续的表示，而解码器则使用这些表示来生成输出序列。

多头注意力（Multi-Head Attention）：Transformer 通过多头注意力机制，允许模型同时从不同的表示子空间捕捉信息，增强了模型的表达能力。

位置编码（Positional Encoding）：由于 Transformer 模型本身不具备捕捉序列顺序的能力，因此引入了位置编码来提供序列中每个元素的位置信息。

层归一化（Layer Normalization）和残差连接（Residual Connections）：Transformer 使用层归一化和残差连接来促进深层网络的训练，防止梯度消失或爆炸问题。

<img src="./images/transformer.jpg" style="zoom:60%;" />


预训练和微调：Transformer 模型通常在大量数据上进行预训练，学习通用的语言表示，然后可以在特定任务上进行微调。

广泛的应用：Transformer 不仅在机器翻译上表现出色，还被广泛应用于其他 NLP 任务，如文本摘要、问答系统、文本分类等。

变体和发展：自原始 Transformer 提出以来，出现了许多改进和变体，如 BERT（Bidirectional Encoder Representations from Transformers）、GPT（Generative Pre-trained Transformer）等，这些模型在不同的任务上都取得了显著的成果。

Transformer 模型的提出，标志着 NLP 领域的一个重要转折点，它为处理序列数据提供了一种全新的视角和强大的工具。

****

基于Transformer的机器翻译

In [1]:
import math
import torch
import pandas as pd
from torch import nn
from matplotlib import pyplot as plt
from models.transformer_utils import MultiHeadAttention, Encoder, AttentionDecoder
from models.transformer_utils import PositionalEncoding, PositionWiseFFN, AddNorm

1.Transformer编码器

In [2]:
# 单层编码器
class EncoderBlock(nn.Module):
    def __init__(self, key_size, query_size, value_size, num_hiddens, 
                 norm_shape, ffn_num_input, ffn_num_hiddens, 
                 num_heads, dropout, use_bias=False, **kwargs):
        super(EncoderBlock, self).__init__(**kwargs)
        self.attention = MultiHeadAttention(key_size, query_size, value_size, 
                                                num_hiddens, num_heads, dropout, use_bias)
        self.addnorm1 = AddNorm(norm_shape, dropout)  # 添加归一化层1

        self.ffn = PositionWiseFFN(ffn_num_input, ffn_num_hiddens, num_hiddens)
        self.addnorm2 = AddNorm(norm_shape, dropout)  # 添加归一化层2
        
    def forward(self, X, valid_lens):
        # 应用多头注意力机制，并将结果加上归一化层1的输出
        Y = self.addnorm1(X, self.attention(X, X, X, valid_lens))
        # 应用位置感知的前馈网络，并将结果加上归一化层2的输出
        return self.addnorm2(Y, self.ffn(Y))  # 返回编码器块的输出

In [3]:
# Transformer编码器
class TransformerEncoder(Encoder):
    def __init__(self, vocab_size, key_size, query_size, value_size, num_hiddens, norm_shape, 
                 ffn_num_input, ffn_num_hiddens, num_heads, 
                 num_layers, dropout, use_bias=False, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.num_hiddens = num_hiddens
        self.embedding = nn.Embedding(vocab_size, num_hiddens)  # 词嵌入层
        self.pos_encoding = PositionalEncoding(num_hiddens, dropout)  # 位置编码层
        self.blks = nn.Sequential()  # 创建一个序列容器来存放编码器块
        
        # 添加编码器块,EncoderBlock是编码器块的实现，参数定义了块的配置
        for i in range(num_layers):
            self.blks.add_module("block"+str(i),
                EncoderBlock(key_size, query_size, value_size, num_hiddens, 
                             norm_shape, ffn_num_input, ffn_num_hiddens, num_heads, dropout, use_bias))

    def forward(self, X, valid_lens, *args):
        # 将输入X通过词嵌入层和位置编码层
        X = self.pos_encoding(self.embedding(X) * math.sqrt(self.num_hiddens))
        # 初始化一个列表来存储每一层的注意力权重
        self.attention_weights = [None] * len(self.blks)
        
        # 遍历所有的编码器块
        for i, blk in enumerate(self.blks):
            X = blk(X, valid_lens)  # 将输入X和有效长度传入块中进行处理
            # 存储当前块的注意力权重
            self.attention_weights[i] = blk.attention.attention.attention_weights
        
        return X

nn.Embedding 是一个层（layer），用于将离散的词汇表（通常由整数索引表示）嵌入到一个密集的向量空间中。这种嵌入常用于处理诸如单词、字符或其他类型的标记的序列数据。

nn.Embedding(vocab_size, num_hiddens) 的参数：

vocab_size：这是词汇表的大小，即索引的不同可能值的数量。例如，如果你有一个由 10000 个唯一单词组成的词汇表，vocab_size 就是 10000。

num_hiddens：这是每个嵌入向量的维度。这个参数决定了嵌入空间的密度。较大的 num_hiddens 可以提供更多的信息容量，但同时也可能导致更复杂的模型，需要更多的数据来训练，并且可能会增加计算成本。

nn.Embedding 层创建后，每个索引值（从 0 到 vocab_size - 1）都会被映射到一个 num_hiddens 维的空间中。这些嵌入向量是通过学习得到的，并且在训练过程中会不断更新。

2.Transformer解码器

In [4]:
# 单层解码器
class DecoderBlock(nn.Module):
    def __init__(self, key_size, query_size, value_size, num_hiddens, 
                 norm_shape, ffn_num_input, ffn_num_hiddens, num_heads, dropout, i, **kwargs):
        super(DecoderBlock, self).__init__(**kwargs)
        self.i = i  # 当前解码器块的索引
        self.attention1 = MultiHeadAttention(key_size, query_size, value_size, 
                                                 num_hiddens, num_heads, dropout)
        self.addnorm1 = AddNorm(norm_shape, dropout)  # 添加归一化层1
        self.attention2 = MultiHeadAttention(key_size, query_size, value_size, 
                                                 num_hiddens, num_heads, dropout)
        self.addnorm2 = AddNorm(norm_shape, dropout)  # 添加归一化层2
        self.ffn = PositionWiseFFN(ffn_num_input, ffn_num_hiddens, num_hiddens)
        self.addnorm3 = AddNorm(norm_shape, dropout)  # 添加归一化层3
        
    def forward(self, X, state):
        enc_outputs, enc_valid_lens = state[0], state[1]  # 从状态中获取编码器的输出和有效长度
        
        # 根据当前解码器块的索引i，决定key_values是当前的X还是之前存储的key_values和当前X的拼接
        if state[2][self.i] is None:
            key_values = X
        else:
            key_values = torch.cat((state[2][self.i], X), axis=1)
        
        state[2][self.i] = key_values  # 更新状态中的key_values
        
        # 根据是否处于训练模式，设置解码器的有效长度
        if self.training:  # 判断是否在训练模式
            batch_size, num_steps, _ = X.shape
            dec_valid_lens = torch.arange(1, num_steps + 1, device=X.device).repeat(batch_size, 1)
        else:
            dec_valid_lens = None
        
        # 应用第一个多头注意力机制
        X2 = self.attention1(X, key_values, key_values, dec_valid_lens)
        # 应用第一个添加归一化层
        Y = self.addnorm1(X, X2)
        # 应用第二个多头注意力机制，这里使用编码器的输出作为键和值
        Y2 = self.attention2(Y, enc_outputs, enc_outputs, enc_valid_lens)
        # 应用第二个添加归一化层
        Z = self.addnorm2(Y, Y2)
        
        # 应用位置感知的前馈网络，并用第三个添加归一化层进行归一化
        return self.addnorm3(Z, self.ffn(Z)), state  # 返回解码器块的输出和状态

In [5]:
class TransformerDecoder(AttentionDecoder):
    def __init__(self, vocab_size, key_size, query_size, value_size, num_hiddens, 
                 norm_shape, ffn_num_input, ffn_num_hiddens, num_heads, num_layers, dropout, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        # 定义解码器的属性
        self.num_hiddens = num_hiddens
        self.num_layers = num_layers
        self.embedding = nn.Embedding(vocab_size, num_hiddens)  # 词嵌入层
        self.pos_encoding = PositionalEncoding(num_hiddens, dropout)  # 位置编码层
        self.blks = nn.Sequential()  # 创建一个序列容器来存放解码器块
        
        # 为每个解码器层创建一个解码器块
        for i in range(num_layers):
            self.blks.add_module("block" + str(i), DecoderBlock(
                key_size, query_size, value_size, num_hiddens, norm_shape, 
                ffn_num_input, ffn_num_hiddens, num_heads, dropout, i))
        
        self.dense = nn.Linear(num_hiddens, vocab_size)  # 输出层，将隐藏层映射到词汇表大小的维度

    def init_state(self, enc_outputs, enc_valid_lens, *args):
        # 初始化解码器的状态
        return [enc_outputs, enc_valid_lens, [None] * self.num_layers]
    
    def forward(self, X, state):
        # 将输入X通过词嵌入层和位置编码层
        X = self.pos_encoding(self.embedding(X) * math.sqrt(self.num_hiddens))
        # 初始化注意力权重矩阵
        self._attention_weights = [[None] * len(self.blks) for _ in range(2)]
        # 遍历所有的解码器块
        for i, blk in enumerate(self.blks):
            X, state = blk(X, state)  # 执行解码器块的前向传播
            # 存储解码器块中第二个注意力机制的权重
            self._attention_weights[1][i] = blk.attention2.attention.attention_weights
        
        # 通过输出层得到预测结果
        return self.dense(X), state
    
    @property
    def attention_weights(self):
        # 提供一个属性来访问解码器的注意力权重
        return self._attention_weights

@property 是一个装饰器，用于创建只读属性的方法，或者更准确地说，是将一个方法变为属性访问的形式。使用 @property 可以让对象的接口看起来更像是使用属性而不是方法，即使底层实现是一个方法。

3.Transformer模型

In [6]:
# 设置模型和训练的超参数
num_hiddens, num_layers, dropout, batch_size, num_steps = 32, 2, 0.1, 64, 10
# 学习率和训练轮数
lr, num_epochs = 0.005, 200

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 定义前馈网络和注意力机制的参数
ffn_num_input, ffn_num_hiddens, num_heads = 32, 64, 4
# 定义键、查询和值的维度
key_size, query_size, value_size = 32, 32, 32
# 定义层归一化操作的形状
norm_shape = [32]

In [7]:
src_vocab_len = 184
tgt_vocab_len = 201

# 实例化Transformer编码器
encoder = TransformerEncoder(
    src_vocab_len, key_size, query_size, value_size, num_hiddens,
    norm_shape, ffn_num_input, ffn_num_hiddens, num_heads, num_layers, dropout)

# 实例化Transformer解码器
decoder = TransformerDecoder(
    tgt_vocab_len, key_size, query_size, value_size, num_hiddens,
    norm_shape, ffn_num_input, ffn_num_hiddens, num_heads, num_layers, dropout)

In [8]:
class Transformer(nn.Module):
    """编码器-解码器架构"""
    def __init__(self, encoder, decoder, **kwargs):
        super(Transformer, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
    def forward(self, enc_X, dec_X, *args):
        enc_outputs = self.encoder(enc_X, *args)
        dec_state = self.decoder.init_state(enc_outputs, *args)
        return self.decoder(dec_X, dec_state)

In [9]:
model = Transformer(encoder, decoder)
print(model)

Transformer(
  (encoder): TransformerEncoder(
    (embedding): Embedding(184, 32)
    (pos_encoding): PositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (blks): Sequential(
      (block0): EncoderBlock(
        (attention): MultiHeadAttention(
          (attention): DotProductAttention(
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (W_q): Linear(in_features=32, out_features=32, bias=False)
          (W_k): Linear(in_features=32, out_features=32, bias=False)
          (W_v): Linear(in_features=32, out_features=32, bias=False)
          (W_o): Linear(in_features=32, out_features=32, bias=False)
        )
        (addnorm1): AddNorm(
          (dropout): Dropout(p=0.1, inplace=False)
          (ln): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        )
        (ffn): PositionWiseFFN(
          (dense1): Linear(in_features=32, out_features=64, bias=True)
          (relu): ReLU()
          (dense2): Linear(in_features=64, out_f